In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import scipy.stats as stats
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import wilcoxon
from scipy.stats import describe
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
import sklearn.metrics
import seaborn as sns
from upsetplot import UpSet

In [152]:
hs_ssal_tpm = pd.read_csv('../Data/deseq_hsdata/ssal_tpm.tsv', sep='\t')
hs_psal_tpm = pd.read_csv('../Data/deseq_hsdata/psal_tpm.tsv', sep='\t')
hs_srsem_tpm = pd.read_csv('../Data/deseq_hsdata/srsem_tpm.tsv', sep='\t')
hs_sfc_tpm = pd.read_csv('../Data/deseq_hsdata/sfc_tpm.tsv', sep='\t')
hs_hfc_tpm = pd.read_csv('../Data/deseq_hsdata/hfc_tpm.tsv', sep='\t')
hs_tpm = hs_ssal_tpm.merge(hs_srsem_tpm.merge(hs_sfc_tpm.merge(hs_hfc_tpm.merge(hs_psal_tpm, on='gene_id', how='outer', suffixes=('', '_psal')), on='gene_id', how='outer', suffixes=('', '_hfc')), on='gene_id', how='outer', suffixes=('', '_sfc')), on='gene_id', how='outer', suffixes=('_ssal', '_srsem'))
hs_tpm = hs_tpm.rename(columns={'gene_name_ssal':'gene_name'}).drop(['gene_name_srsem', 'gene_name_hfc', 'gene_name_psal', 'transcript_id(s)'], axis=1)

In [151]:
# read merged tpm dataframes
#hs_tpm = pd.read_csv('../Data/hs_conc_tpm.tsv', sep='\t')
#at_tpm = pd.read_csv('../Data/at_conc_tpm.tsv', sep='\t')
#dr_tpm = pd.read_csv('../Data/dr_conc_tpm.tsv', sep='\t')
at_ssal_tpm = pd.read_csv('../Data/deseq_tairdata/ssal_tpm.tsv', sep='\t')
at_psal_tpm = pd.read_csv('../Data/deseq_tairdata/psal_tpm.tsv', sep='\t')
at_srsem_tpm = pd.read_csv('../Data/deseq_tairdata/srsem_tpm.tsv', sep='\t')
at_sfc_tpm = pd.read_csv('../Data/deseq_tairdata/sfc_tpm.tsv', sep='\t')
at_hfc_tpm = pd.read_csv('../Data/deseq_tairdata/hfc_tpm.tsv', sep='\t')
at_tpm = at_ssal_tpm.merge(at_srsem_tpm.merge(at_sfc_tpm.merge(at_hfc_tpm.merge(at_psal_tpm, on='gene_id', how='outer', suffixes=('', '_psal')), on='gene_id', how='outer', suffixes=('', '_hfc')), on='gene_id', how='outer', suffixes=('', '_sfc')), on='gene_id', how='outer', suffixes=('_ssal', '_srsem'))
at_tpm = at_tpm.rename(columns={'gene_name_ssal':'gene_name'}).drop(['gene_name_srsem', 'gene_name_psal', 'gene_name_hfc','transcript_id(s)'], axis=1)

In [150]:
dr_colnames = {'SRR3655791':'sham11', 'SRR3655792':'sham12', 'SRR3655793':'sham21',
       'SRR3655794':'sham22', 'SRR3655795':'sham31', 'SRR3655796':'sham32', 'SRR3655797':'sbs11', 'SRR3655798':'sbs12',
       'SRR3655799':'sbs21', 'SRR3655800':'sbs22', 'SRR3655801':'sbs31', 'SRR3655802':'sbs32'}
dr_ssal_tpm = pd.read_csv('../Data/deseq_daniodata/ssal_tpm.tsv', sep='\t').rename(columns=dr_colnames)
dr_psal_tpm = pd.read_csv('../Data/deseq_daniodata/psal_tpm.tsv', sep='\t').rename(columns=dr_colnames)
dr_srsem_tpm = pd.read_csv('../Data/deseq_daniodata/srsem_tpm.tsv', sep='\t').rename(columns=dr_colnames)
dr_sfc_tpm = pd.read_csv('../Data/deseq_daniodata/sfc_tpm.tsv', sep='\t')
dr_hfc_tpm = pd.read_csv('../Data/deseq_daniodata/hfc_tpm.tsv', sep='\t')
dr_tpm = dr_ssal_tpm.merge(dr_srsem_tpm.merge(dr_sfc_tpm.merge(dr_hfc_tpm.merge(dr_psal_tpm, on='gene_id', how='outer', suffixes=('', '_psal')), on='gene_id', how='outer', suffixes=('', '_hfc')), on='gene_id', how='outer', suffixes=('', '_sfc')), on='gene_id', how='outer', suffixes=('_ssal', '_srsem'))
dr_tpm = dr_tpm.rename(columns={'gene_name_ssal':'gene_name'}).drop(['gene_name_srsem', 'gene_name_hfc', 'gene_name_psal', 'transcript_id(s)'], axis=1)

In [22]:
hs_tpmcols = ['A1_ssal', 'A2_ssal','A3_ssal', 'A4_ssal', 'A5_ssal', 'B1_ssal', 'B2_ssal', 'B3_ssal',
       'B4_ssal', 'B5_ssal','A1_srsem', 'A2_srsem',
       'A3_srsem', 'A4_srsem', 'A5_srsem', 'B1_srsem', 'B2_srsem', 'B3_srsem',
       'B4_srsem', 'B5_srsem', 'A1_sfc', 'A2_sfc', 'A3_sfc', 'A4_sfc',
       'A5_sfc', 'B1_sfc', 'B2_sfc', 'B3_sfc', 'B4_sfc', 'B5_sfc', 'A1_hfc',
       'A2_hfc', 'A3_hfc', 'A4_hfc', 'A5_hfc', 'B1_hfc', 'B2_hfc', 'B3_hfc',
       'B4_hfc', 'B5_hfc', 'A1_psal', 'A2_psal', 'A3_psal', 'A4_psal',
       'A5_psal', 'B1_psal', 'B2_psal', 'B3_psal', 'B4_psal', 'B5_psal']
at_tpmcols = ['flight1_ssal', 'flight2_ssal','flight3_ssal', 'ground1_ssal', 'ground2_ssal', 'ground3_ssal',
       'flight1_srsem', 'flight2_srsem', 'flight3_srsem',
       'ground1_srsem', 'ground2_srsem', 'ground3_srsem', 'flight1_sfc',
       'flight2_sfc', 'flight3_sfc', 'ground1_sfc', 'ground2_sfc',
       'ground3_sfc', 'flight1_hfc', 'flight2_hfc', 'flight3_hfc',
       'ground1_hfc', 'ground2_hfc', 'ground3_hfc', 'flight1_psal',
       'flight2_psal', 'flight3_psal', 'ground1_psal', 'ground2_psal',
       'ground3_psal']
dr_tpmcols = ['sham11_ssal', 'sham12_ssal','sham21_ssal', 'sham22_ssal', 'sham31_ssal', 'sham32_ssal',
       'sbs11_ssal', 'sbs12_ssal', 'sbs21_ssal', 'sbs22_ssal', 'sbs31_ssal',
       'sbs32_ssal', 'sham11_srsem', 'sham12_srsem',
       'sham21_srsem', 'sham22_srsem', 'sham31_srsem', 'sham32_srsem',
       'sbs11_srsem', 'sbs12_srsem', 'sbs21_srsem', 'sbs22_srsem',
       'sbs31_srsem', 'sbs32_srsem', 'sham11_sfc', 'sham12_sfc', 'sham21_sfc',
       'sham22_sfc', 'sham31_sfc', 'sham32_sfc', 'sbs11_sfc', 'sbs12_sfc',
       'sbs21_sfc', 'sbs22_sfc', 'sbs31_sfc', 'sbs32_sfc', 'sham11_hfc',
       'sham12_hfc', 'sham21_hfc', 'sham22_hfc', 'sham31_hfc', 'sham32_hfc',
       'sbs11_hfc', 'sbs12_hfc', 'sbs21_hfc', 'sbs22_hfc', 'sbs31_hfc',
       'sbs32_hfc', 'sham11_psal', 'sham12_psal', 'sham21_psal', 'sham22_psal',
       'sham31_psal', 'sham32_psal', 'sbs11_psal', 'sbs12_psal', 'sbs21_psal',
       'sbs22_psal', 'sbs31_psal', 'sbs32_psal']

In [167]:
hs_tpm_mean = hs_tpm[hs_tpmcols].stack().mean()
at_tpm_mean = at_tpm[at_tpmcols].stack().mean()
dr_tpm_mean = dr_tpm[dr_tpmcols].stack().mean()

hs_tpm_median = hs_tpm[hs_tpmcols].stack().median()
at_tpm_median = at_tpm[at_tpmcols].stack().median()
dr_tpm_median = dr_tpm[dr_tpmcols].stack().median()

In [168]:
print('MEAN hs: {:.5}, at: {:.5}, dr: {:.5}'.format(hs_tpm_mean, at_tpm_mean, dr_tpm_mean))

MEAN hs: 16.628, at: 29.781, dr: 21.987


In [169]:
print('MEDIAN hs: {:.5}, at: {:.5}, dr: {:.5}'.format(hs_tpm_median, at_tpm_median, dr_tpm_median))

MEDIAN hs: 0.023274, at: 2.8996, dr: 0.15913


In [145]:
# Get high expressed genes
# genes which in at least one of the columns have a TPM greater than the mean/median
#df.loc[(df > mean).any(axis=1)]

def genelist(df, m):
    df = df.drop('gene_name', axis=1).set_index('gene_id')
    df = df.loc[(df > m).any(axis=1)]
    genelist = list(df.index.values)
    return df, genelist

In [155]:
hs_hiexgene_mean_df, hs_hiexgene_mean_lst = genelist(hs_tpm, hs_tpm_mean)
hs_hiexgene_median_df, hs_hiexgene_median_lst = genelist(hs_tpm, hs_tpm_median)

at_hiexgene_mean_df, at_hiexgene_mean_lst = genelist(at_tpm, at_tpm_mean)
at_hiexgene_median_df, at_hiexgene_median_lst = genelist(at_tpm, at_tpm_median)

dr_hiexgene_mean_df, dr_hiexgene_mean_lst = genelist(dr_tpm, dr_tpm_mean)
dr_hiexgene_median_df, dr_hiexgene_median_lst = genelist(dr_tpm, dr_tpm_median)


In [166]:
print('Above MEAN')
print('hs: {} of {} genes with TPM greater than {:.5}'.format(hs_hiexgene_mean_df.shape[0], hs_tpm.shape[0], hs_tpm_mean))
print('at: {} of {} genes with TPM greater than {:.5}'.format(at_hiexgene_mean_df.shape[0], at_tpm.shape[0], at_tpm_mean))
print('dr: {} of {} genes with TPM greater than {:.5}'.format(dr_hiexgene_mean_df.shape[0], dr_tpm.shape[0], dr_tpm_mean))

print('Above MEDIAN')
print('hs: {} of {} genes with TPM greater than {:.5}'.format(hs_hiexgene_median_df.shape[0], hs_tpm.shape[0], hs_tpm_median))
print('at: {} of {} genes with TPM greater than {:.5}'.format(at_hiexgene_median_df.shape[0], at_tpm.shape[0], at_tpm_median))
print('dr: {} of {} genes with TPM greater than {:.5}'.format(dr_hiexgene_median_df.shape[0], dr_tpm.shape[0], dr_tpm_median))


Above MEAN
hs: 9992 of 63769 genes with TPM greater than 16.628
at: 7710 of 33694 genes with TPM greater than 29.781
dr: 7576 of 47143 genes with TPM greater than 21.987
Above MEDIAN
hs: 47427 of 63769 genes with TPM greater than 0.023274
at: 19121 of 33694 genes with TPM greater than 2.8996
dr: 31875 of 47143 genes with TPM greater than 0.15913


In [175]:
# write gene list to file
with open(r'../Data/dr_hiexpr_lst.txt', 'w') as fp:
    for item in dr_hiexgene_mean_lst:
        # write each item on a new line
        fp.write("%s\n" % item)

In [157]:
describe(hs_tpm[hs_tpmcols].stack())

DescribeResult(nobs=3006910, minmax=(0.0, 774786.5), mean=16.628343826687207, variance=3590007.8581219157, skewness=376.37316621469654, kurtosis=145351.99713218026)

In [158]:
describe(at_tpm[at_tpmcols].stack())

DescribeResult(nobs=1007370, minmax=(0.0, 79521.655596), mean=29.780519551238374, variance=191602.34089137646, skewness=126.38642762925518, kurtosis=18781.2553287411)

In [159]:
describe(dr_tpm[dr_tpmcols].stack())

DescribeResult(nobs=2728944, minmax=(0.0, 45165.839844), mean=21.986522002296855, variance=94012.73168713611, skewness=50.8343776665543, kurtosis=4059.149402215809)

In [ ]:
#plt.boxplot(hs_tpm[hs_tpmcols].stack())
#plt.boxplot(at_tpm[at_tpmcols].stack())
plt.violinplot(dr_tpm[dr_tpmcols].stack())

In [13]:
# read DEseq dataframes
hs_de = pd.read_csv('../Data/hs_df_all.tsv', sep='\t')
at_de = pd.read_csv('../Data/at_df_all.tsv', sep='\t')
dr_de = pd.read_csv('../Data/dr_df_all.tsv', sep='\t')